In [1]:
import json
import csv
from data import Modeling

In [2]:
def make_json(csvFilePath, jsonFilePath,primaryKey):
     
    # create a dictionary
    data = {}
     
    # Open a csv reader called DictReader
    with open(csvFilePath, encoding='utf-8') as csvf:
        csvReader = csv.DictReader(csvf)
         
        # Convert each row into a dictionary 
        # and add it to data
        for rows in csvReader:
             
            # Assuming a column primaryKey to
            # be the primary key
            key = rows[primaryKey]
            data[key] = rows
 
    # Open a json writer, and use the json.dumps() 
    # function to dump data
    with open(jsonFilePath, 'w', encoding='utf-8') as jsonf:
        jsonf.write(json.dumps(data, indent=4))

In [12]:
make_json("Datassets/tested.csv","Datassets/titanic.json","PassengerId")

In [3]:
with open("Datassets/titanic.json","r") as file:
    datasset = json.load(file)

In [4]:
datasset = [datasset[i] for i in datasset]

In [8]:
datasset[0].keys()

dict_keys(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'])

In [10]:
titatic_data = Modeling(datasset,['Survived'],drops=["Cabin","Name"]) # Retirando colunas que não servem

In [55]:
titatic_data.dataframe["Embarked"].unique()

array(['Q', 'S', 'C'], dtype=object)

# Aplicando o modelo:

In [17]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

In [18]:

# Create a Random Forest Classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
 
# Train the classifier
rf_classifier.fit(titatic_data.X_train, titatic_data.y_train)
 
# Make predictions on the test set
y_pred = rf_classifier.predict(titatic_data.X_test)
 
# Evaluate the model
accuracy = accuracy_score(titatic_data.y_test, y_pred)
classification_rep = classification_report(titatic_data.y_test, y_pred)
 
# Print the results
print(f"Accuracy: {accuracy:.2f}")
print("\nClassification Report:\n", classification_rep)

Accuracy: 1.00

Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        55
           1       1.00      1.00      1.00        29

    accuracy                           1.00        84
   macro avg       1.00      1.00      1.00        84
weighted avg       1.00      1.00      1.00        84



c:\Users\lknmu\OneDrive\Área de Trabalho\Projetos\GitHub\ML-Hub\Ambiente-de-desenvolvimento\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


In [89]:
titatic_data.encoded_features

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,310,2,1,40,0,0,152,140,1
1,311,2,0,56,1,0,221,121,2
2,312,1,1,72,0,0,73,167,1
3,313,2,1,29,0,0,147,157,2
4,314,2,0,22,1,1,138,8,2
...,...,...,...,...,...,...,...,...,...
413,305,2,1,0,0,0,267,154,2
414,306,0,0,47,0,0,324,5,0
415,307,2,1,46,0,0,346,125,2
416,308,2,1,0,0,0,220,154,2


In [90]:
titatic_data.dataframe

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,892,0,3,male,34.5,0,0,330911,7.8292,Q
1,893,1,3,female,47,1,0,363272,7,S
2,894,0,2,male,62,0,0,240276,9.6875,Q
3,895,0,3,male,27,0,0,315154,8.6625,S
4,896,1,3,female,22,1,1,3101298,12.2875,S
...,...,...,...,...,...,...,...,...,...,...
413,1305,0,3,male,,0,0,A.5. 3236,8.05,S
414,1306,1,1,female,39,0,0,PC 17758,108.9,C
415,1307,0,3,male,38.5,0,0,SOTON/O.Q. 3101262,7.25,S
416,1308,0,3,male,,0,0,359309,8.05,S


In [21]:
import pandas as pd

In [98]:
# Assuming 'data' is your dictionary containing the data
data = {
    "PassengerId": [1400],
    "Pclass": [3],
    "Sex": ["female"],
    "Age": [80],
    "SibSp": [1],
    "Parch": [0],
    "Ticket": ["330914"],
    "Fare": [7.8292],
    "Embarked": ["S"]
}



In [99]:
# Define index and columns for DataFrame
index = [0]  # Assuming a single row DataFrame
columns = list(data.keys())

# Create DataFrame
df = pd.DataFrame(data, index=index, columns=columns)

In [100]:
df.loc[:, 'Sex'] = df['Sex'].map({'female': 0, 'male': 1})

In [ ]:
'Q', 'S', 'C'

In [101]:
df.loc[:, 'Embarked'] = df['Embarked'].map({'Q': 1, 'S': 2,"C":0})

In [102]:
df

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,1400,3,0,80,1,0,330914,7.8292,2


In [103]:
teste_new_pred = rf_classifier.predict(df)

In [104]:
teste_new_pred

array(['1'], dtype=object)